In [1]:
import pandas as pd

df = pd.read_excel(r'C:\Users\jiamv\Downloads\Low_Conc_Experiments.xlsx')

X = df.iloc[:, [1, 9, 10, 11]]

Y = df.iloc[:, list(range(1, 3))]


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

In [3]:
from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler()
scaler_Y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled  = scaler_X.transform(X_test)

Y_train_scaled = scaler_Y.fit_transform(Y_train)


In [4]:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor

svr = SVR(
    kernel='rbf',   # most common & powerful
    C=100,
    gamma='scale',
    epsilon=0.1
)

model = MultiOutputRegressor(svr)
model.fit(X_train_scaled, Y_train_scaled)


MultiOutputRegressor(estimator=SVR(C=100))

In [6]:
X_full_scaled = scaler_X.transform(X)
Y_full_pred_scaled = model.predict(X_full_scaled)
Y_full_pred = scaler_Y.inverse_transform(Y_full_pred_scaled)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(Y, Y_full_pred)
mse = mean_squared_error(Y, Y_full_pred)
r2  = r2_score(Y, Y_full_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)
print("Support Vectors:", model.estimators_[0].support_vectors_)
print("Support Vector Indices:", model.estimators_[0].support_)
print("Dual Coefficients:", model.estimators_[0].dual_coef_)
print("Intercept:", model.estimators_[0].intercept_)
print("SVM Parameters:", model.estimators_[0].get_params())
print("Regularization (C):", model.estimators_[0].C)
print("Epsilon:", model.estimators_[0].epsilon)
print("Gamma (smoothing):", model.estimators_[0].gamma)

Mean Absolute Error: 3.49061962732896
Mean Squared Error: 14.927813628215388
R-squared: 0.9953459299074268
Support Vectors: [[-1.66225996  2.29494441  1.97777605  2.18790075]
 [-1.09348595  1.49860037  1.76758059  1.48014369]
 [-1.54037981  1.8089489   1.58028899  1.80522517]
 [-1.74351339  0.6981581   2.20626385  0.63047002]
 [-1.68257332  1.6240625   2.01437355  1.84158866]
 [-0.74815888  0.7151277   0.52280754  0.73494785]
 [ 0.9784765  -0.8525293  -0.83208466 -0.8581989 ]
 [ 1.70975737 -0.88996549 -0.85442489 -0.88687957]]
Support Vector Indices: [ 29  30  34  37  75  77 101 129]
Dual Coefficients: [[-1.15489906  1.72581522 -0.64563409 -1.52056578 -0.21509997 -0.05882137
  -1.16044301  3.02964806]]
Intercept: [-0.40313368]
SVM Parameters: {'C': 100, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Regularization (C): 100
Epsilon: 0.1
Gamma (smoothing): scale


In [8]:
X_synthetic = X_train_scaled
Y_svm_pred = model.predict(X_synthetic)

from sklearn.ensemble import RandomForestRegressor
rf_student = RandomForestRegressor(n_estimators=8, max_depth=3)
rf_student.fit(X_synthetic, Y_svm_pred)

from micromlgen import port
with open("SVM_output2.h", "w") as f:
    f.write(port(rf_student))